In [125]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import re

In [4]:
import json

In [126]:
import ast
# LLM-based history update solution
import importlib.util
import inspect
from typing import List
import requests
import json
import re
import difflib
from collections import Counter
# from spellchecker import SpellChecker
from datetime import datetime
import pandas as pd
import ast
import random
import logging 
# from history_update_problem.call_or import export_rows
from call_or import *
from evaluation import *

# Worflow eval

In [189]:
def eval_workflows(pp_id, gt_wf_fp, pred_wf_fp):
    # print(gt_wf_fp)
    gt_ops_list = parse_recipe(pp_id, recipe=gt_wf_fp)
    pred_ops_list = parse_recipe(pp_id, recipe=pred_wf_fp)
    
    return {'pp_id': pp_id, 'gt_ops': gt_ops_list[pp_id], 'pred_ops': pred_ops_list[pp_id]}
    # print(gt_ops_list)
    # print(pred_ops_list)


models = ['llama3.1', 'gemma2', 'mistral']
answer_gt_path = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_small_table.json'
answer_preds_llama = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_llama3.1.json'


# eval_answer_results = eval_answers(answer_gt_path, answer_preds_llama)
model = models[2]
wf_gt_folder = '/projects/bces/lanl2/LLM4DC/datasets'
wf_pred_folder = f'/projects/bces/lanl2/LLM4DC/CoT.response/{model}/recipes_llm'

ops_list = []
query_contents = pd.read_csv('/projects/bces/lanl2/LLM4DC/purposes/queries.csv')
for query_id in range(111):
    row = query_contents[query_contents['ID'] == query_id]
    if len(row) == 0:
        continue
    if model == 'llama3.1':
        if query_id >= 62 and query_id <= 91:
            target_path = f'{wf_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/ppp_datasets/workflows/ppp_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/ppp_test_{query_id}.json"
        elif query_id >= 92:
            target_path = f'{wf_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/dish_datasets/workflows/dish_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/dish_test_{query_id}.json"
        elif query_id >= 31 and query_id <= 61:
            target_path = f'{wf_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/chi_food_inspection_datasets/workflows/chi_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/chi_test_{query_id}.json"
        elif query_id <31:
            target_path = f'{wf_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}'
            wf_gt_fp = f"{wf_gt_folder}/purpose-prepared-datasets/menu/workflows/menu_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/menu_test_{query_id}.json"
    else: 
        if query_id >= 62 and query_id <= 91:
            target_path = f'{wf_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/ppp_datasets/workflows/ppp_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_ppp_test_{query_id}.json"
        elif query_id >= 92:
            target_path = f'{wf_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/dish_datasets/workflows/dish_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_dish_test_{query_id}.json"
        elif query_id >= 31 and query_id <= 61:
            target_path = f'{wf_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/chi_food_inspection_datasets/workflows/chi_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_chi_test_{query_id}.json"
        elif query_id <31:
            target_path = f'{wf_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}'
            wf_gt_fp = f"{wf_gt_folder}/purpose-prepared-datasets/menu/workflows/menu_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_menu_test_{query_id}.json"

    ops_list.append(eval_workflows(query_id, wf_gt_fp, wf_pred_fp))
ops_df = pd.DataFrame(ops_list)

In [190]:
ops_df['gt_ops_length'] = ops_df['gt_ops'].apply(len)
ops_df['pred_ops_length'] = ops_df['pred_ops'].apply(len)

In [191]:
ops_df['gt_ops_set_length'] = ops_df['gt_ops'].apply(lambda x: len(set(x)))
ops_df['pred_ops_set_length'] = ops_df['pred_ops'].apply(lambda x: len(set(x)))

In [399]:
ops_df.describe()

,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,67.000000,67.000000,67.000000,67.000000,67.000000
mean,58.074627,5.268657,3.402985,2.925373,1.955224
std,35.047290,3.226698,2.322875,1.197389,0.944407
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,31.500000,3.000000,2.000000,2.000000,1.000000
50%,66.000000,5.000000,3.000000,3.000000,2.000000
75%,88.000000,6.000000,5.000000,4.000000,2.000000
max,110.000000,14.000000,10.000000,4.000000,4.000000


In [6]:
from evaluation import calculate_operation_metrics

In [192]:
workflow_results = calculate_operation_metrics(ops_df['gt_ops'], ops_df['pred_ops'])
workflow_results['pp_id'] = ops_df['pp_id']

In [193]:
workflow_results.to_csv(f'evaluation/workflow_result_{model}.csv')

In [182]:
workflow_results.describe()

,accuracy,precision,recall,f1,pp_id
count,67.000000,67.000000,67.000000,67.000000,67.000000
mean,0.238806,0.855721,0.615672,0.684151,58.074627
std,0.429572,0.241254,0.282767,0.246258,35.047290
min,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.750000,0.500000,0.500000,31.500000
50%,0.000000,1.000000,0.500000,0.666667,66.000000
75%,0.000000,1.000000,0.875000,0.857143,88.000000
max,1.000000,1.000000,1.000000,1.000000,110.000000


In [387]:
ppp_results = workflow_results[workflow_results['pp_id'] >= 62][workflow_results['pp_id'] <=91]
ppp_results.describe()

/tmp/ipykernel_671706/814825836.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_results = workflow_results[workflow_results['pp_id'] >= 62][workflow_results['pp_id'] <=91]


,accuracy,precision,recall,f1,pp_id
count,22.000000,22.000000,22.000000,22.000000,22.000000
mean,0.272727,0.799242,0.757576,0.734632,73.000000
std,0.455842,0.284822,0.285112,0.235405,7.438638
min,0.000000,0.000000,0.000000,0.000000,62.000000
25%,0.000000,0.666667,0.541667,0.666667,67.250000
50%,0.000000,1.000000,0.833333,0.666667,72.500000
75%,0.750000,1.000000,1.000000,0.964286,77.750000
max,1.000000,1.000000,1.000000,1.000000,89.000000


In [388]:
ppp_ops = ops_df[ops_df['pp_id'] >= 62][ops_df['pp_id'] <=91]
ppp_ops.describe()


/tmp/ipykernel_671706/3831109375.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_ops = ops_df[ops_df['pp_id'] >= 62][ops_df['pp_id'] <=91]


,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,22.000000,22.000000,22.000000,22.000000,22.000000
mean,73.000000,3.409091,3.863636,2.363636,2.136364
std,7.438638,1.842852,2.816541,1.255292,0.940894
min,62.000000,1.000000,1.000000,1.000000,1.000000
25%,67.250000,2.000000,2.000000,1.000000,1.250000
50%,72.500000,3.000000,3.000000,3.000000,2.000000
75%,77.750000,5.000000,4.000000,3.000000,3.000000
max,89.000000,6.000000,10.000000,4.000000,4.000000


In [389]:
dish_results = workflow_results[workflow_results['pp_id'] >= 92]
dish_results.describe()

,accuracy,precision,recall,f1,pp_id
count,16.0,16.000000,16.000000,16.000000,16.00000
mean,0.0,0.760417,0.515625,0.596429,101.93750
std,0.0,0.261539,0.213478,0.207858,5.65059
min,0.0,0.000000,0.000000,0.000000,92.00000
25%,0.0,0.666667,0.500000,0.571429,98.75000
50%,0.0,0.750000,0.500000,0.666667,102.50000
75%,0.0,1.000000,0.750000,0.750000,106.25000
max,0.0,1.000000,0.750000,0.857143,110.00000


In [391]:
dish_ops= ops_df[ops_df['pp_id'] >= 92]

dish_ops.describe()

,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,16.00000,16.000000,16.000000,16.0000,16.000000
mean,101.93750,5.375000,5.625000,3.8125,2.687500
std,5.65059,1.310216,2.552776,0.7500,1.014479
min,92.00000,1.000000,1.000000,1.0000,1.000000
25%,98.75000,5.000000,3.750000,4.0000,2.000000
50%,102.50000,6.000000,6.000000,4.0000,3.000000
75%,106.25000,6.000000,8.000000,4.0000,3.250000
max,110.00000,7.000000,9.000000,4.0000,4.000000


In [401]:
chi_results = workflow_results[workflow_results['pp_id'] >= 31][workflow_results['pp_id'] <=61]
chi_results.describe()

/tmp/ipykernel_671706/2907689086.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_results = workflow_results[workflow_results['pp_id'] >= 31][workflow_results['pp_id'] <=61]


,accuracy,precision,recall,f1,pp_id
count,13.000000,13.000000,13.000000,13.000000,13.000000
mean,0.153846,0.692308,0.506410,0.538462,38.769231
std,0.375534,0.389956,0.347713,0.314126,6.260376
min,0.000000,0.000000,0.000000,0.000000,31.000000
25%,0.000000,0.500000,0.333333,0.400000,34.000000
50%,0.000000,1.000000,0.500000,0.500000,38.000000
75%,0.000000,1.000000,0.666667,0.666667,41.000000
max,1.000000,1.000000,1.000000,1.000000,52.000000


In [393]:
chi_ops = ops_df[ops_df['pp_id'] >= 31][ops_df['pp_id'] <=61]

chi_ops.describe()

/tmp/ipykernel_671706/2470253216.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_ops = ops_df[ops_df['pp_id'] >= 31][ops_df['pp_id'] <=61]


,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,13.000000,13.000000,13.000000,13.000000,13.000000
mean,38.769231,4.769231,5.153846,2.692308,2.230769
std,6.260376,2.047513,3.647971,1.031553,1.091928
min,31.000000,1.000000,1.000000,1.000000,1.000000
25%,34.000000,4.000000,3.000000,2.000000,1.000000
50%,38.000000,5.000000,4.000000,3.000000,2.000000
75%,41.000000,6.000000,9.000000,3.000000,3.000000
max,52.000000,8.000000,13.000000,4.000000,4.000000


In [394]:
menu_results = workflow_results[workflow_results['pp_id'] < 31]
menu_results.describe()

,accuracy,precision,recall,f1,pp_id
count,16.0000,16.000000,16.000000,16.000000,16.000000
mean,0.0625,0.802083,0.510417,0.579464,9.375000
std,0.2500,0.299498,0.280335,0.220044,5.667157
min,0.0000,0.000000,0.000000,0.000000,1.000000
25%,0.0000,0.625000,0.333333,0.500000,4.750000
50%,0.0000,1.000000,0.500000,0.666667,8.500000
75%,0.0000,1.000000,0.500000,0.666667,14.250000
max,1.0000,1.000000,1.000000,1.000000,18.000000


In [395]:
menu_ops = ops_df[ops_df['pp_id'] < 31]
menu_ops.describe()

,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,16.000000,16.000000,16.000000,16.000000,16.0000
mean,9.375000,8.125000,5.625000,3.000000,1.6875
std,5.667157,4.688639,5.389805,1.154701,0.7932
min,1.000000,1.000000,0.000000,1.000000,0.0000
25%,4.750000,4.000000,1.750000,2.000000,1.0000
50%,8.500000,7.500000,3.500000,3.500000,2.0000
75%,14.250000,11.750000,8.500000,4.000000,2.0000
max,18.000000,14.000000,18.000000,4.000000,3.0000


# Dataset Eval

In [44]:
# def retrieve_tg_cols(tg_cols_fp="target_columns_list.csv"):
#     id_tg_cols = {}
#     tg_df = pd.read_csv(tg_cols_fp)
#     result_dict = tg_df.set_index('ID')['tg_columns'].to_dict()
#     return result_dict

In [157]:
result_dict = retrieve_tg_cols("/projects/bces/lanl2/LLM4DC/evaluation/target_columns_list.csv")
print(result_dict)
model = "gemma2"
# model = "dirty"
data_gt_folder = "/projects/bces/lanl2/LLM4DC/datasets"
ratio_list = []
for query_id in range(111):
    # print(query_id)
    tg_cols = result_dict.get(query_id)
    if tg_cols:
        if model == "llama3.1":
            llm_folder = f"CoT.response/{model}/datasets_llm"
            pred_fp = f'/projects/bces/lanl2/LLM4DC/{llm_folder}'
            if query_id >= 62 and query_id <= 91:
                target_path = f'{data_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/ppp_test_{query_id}.csv'

            elif query_id >= 92:
                target_path = f'{data_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/dish_test_{query_id}.csv'

            elif query_id >= 31 and query_id <= 61:
                target_path = f'{data_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/chi_test_{query_id}.csv'

            elif query_id <31:
                target_path = f'{data_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/menu_test_{query_id}.csv'
        elif model=="dirty":
            if query_id >= 62 and query_id <= 91:
                target_path = f'{data_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
                table_preds_path =  f'/projects/bces/lanl2/LLM4DC/datasets/ppp_datasets/ppp_data_p{query_id}.csv'
            elif query_id >= 92:
                target_path = f'{data_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
                table_preds_path = f'/projects/bces/lanl2/LLM4DC/datasets/dish_datasets/dish_data_p{query_id}.csv'
            elif query_id >= 31 and query_id <= 61:
                target_path = f'{data_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
                table_preds_path = f'/projects/bces/lanl2/LLM4DC/datasets/chi_food_inspection_datasets/chi_food_data_p{query_id}.csv'
            elif query_id <31:
                target_path = f'{data_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}.csv'
                table_preds_path = f'/projects/bces/lanl2/LLM4DC/datasets/purpose-prepared-datasets/menu/menu_data.csv'
        else:

            llm_folder = f"CoT.response/{model}/datasets_llm"
            data_gt_folder = "/projects/bces/lanl2/LLM4DC/datasets"
            pred_fp = f'/projects/bces/lanl2/LLM4DC/{llm_folder}'
            if query_id >= 62 and query_id <= 91:
                target_path = f'{data_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_ppp_test_{query_id}.csv'

            elif query_id >= 92:
                target_path = f'{data_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_dish_test_{query_id}.csv'

            elif query_id >= 31 and query_id <= 61:
                target_path = f'{data_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_chi_test_{query_id}.csv'

            elif query_id <31:
                target_path = f'{data_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_menu_test_{query_id}.csv'
        gt_df = pd.read_csv(target_path)
        preds_df = pd.read_csv(table_preds_path)
        # print(gt_df.head(5), preds_df.head(5))
        res = average_match_ratio(gt_df, preds_df, tg_cols)
        ratio_list.append({'pp_id': query_id, 'ratio': res})
dt_result = pd.DataFrame(ratio_list)

{1: 'page_count', 2: 'page_count', 3: 'event', 4: 'event', 5: 'event', 6: 'venue', 7: 'occasion', 8: 'occasion', 9: 'page_count, dish_count', 12: 'page_count, location', 13: 'sponsor, currency', 14: 'sponsor, dish_count', 15: 'sponsor, event', 16: 'sponsor, event', 17: 'sponsor, event', 18: 'sponsor, event', 31: 'Risk', 32: 'Results', 33: 'Facility Type', 34: 'Facility Type', 36: 'DBA Name, Results', 37: 'DBA Name, Results', 38: 'Facility Type, Risk', 39: 'Facility Type, Risk', 40: 'Facility Type, Risk', 41: 'Facility Type, Risk', 42: 'Facility Type, Risk', 49: 'Facility Type, Risk, Results', 52: 'Address, Risk, Results', 62: 'LoanAmount', 63: 'LoanAmount', 64: 'LoanAmount', 65: 'NAICSCode, JobsReported', 66: 'JobsReported, LoanAmount', 67: 'City', 68: 'BusinessType, LoanAmount', 69: 'BusinessType, LoanAmount', 70: 'BusinessType, LoanAmount', 71: 'BusinessType, Zip', 72: 'Zip, LoanAmount', 73: 'Zip, LoanAmount', 74: 'Gender, LoanAmount', 75: 'Gender, LoanAmount', 76: 'City, LoanAmount'

In [158]:
model

'gemma2'

In [159]:
dt_result.to_csv(f'evaluation/{model}_table_result.csv')

In [151]:
dt_result.head()

,pp_id,ratio
0,1,0.30
1,2,1.00
2,3,0.26
3,4,0.26
4,5,0.26


In [160]:
stat = dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.733834237901402
0.24074387348355658
0.21
1.0
0.5925
0.775
0.9708333333333333


In [161]:
ppp_dt_results = dt_result[dt_result['pp_id'] >= 62][dt_result['pp_id'] <=91]
stat = ppp_dt_results['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.8672348484848484
0.16225457379430547
0.5
1.0
0.7375
0.9708333333333333
1.0


/tmp/ipykernel_1577642/1140604443.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_dt_results = dt_result[dt_result['pp_id'] >= 62][dt_result['pp_id'] <=91]


In [162]:
dish_dt_result =dt_result[dt_result['pp_id'] >= 92]
stat = dish_dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.7523958333333333
0.12962048413506025
0.5
1.0
0.6725
0.7566666666666666
0.8066666666666666


In [164]:
chi_dt_result =dt_result[dt_result['pp_id'] >= 31][dt_result['pp_id'] <=61]
stat = chi_dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.6607226107226107
0.2738546155369595
0.25
1.0
0.45
0.7727272727272727
0.9


/tmp/ipykernel_1577642/3331208062.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_dt_result =dt_result[dt_result['pp_id'] >= 31][dt_result['pp_id'] <=61]


In [165]:
menu_dt_result =dt_result[dt_result['pp_id'] < 31]

stat = menu_dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.59125
0.2976099236696698
0.21
1.0
0.315
0.5925
0.935


## table eval ttest

# answer eval

In [55]:
datafile_path = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_gt.json'
data = []
with open(datafile_path, 'r') as f:
    for l in f:
        data.append(json.loads(l))

In [6]:
from bert_score import score

/u/lirif2/.conda/envs/llm4dc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
import json
from typing import Union, List, Dict, Any
from difflib import SequenceMatcher
from math import isclose

# Define a utility to convert JSON strings to Python objects
def parse_input(answer: Union[str, float, List, Dict]) -> Any:
    if isinstance(answer, str):
        try:
            # Try to parse JSON strings into Python objects
            return json.loads(answer)
        except json.JSONDecodeError:
            return answer.lower().strip()  # Normalize strings for comparison
    elif isinstance(answer, float):
        return round(answer, 2)  # Round floats to two decimal places if needed
    return answer  # If already in desired format

# Calculate exact match accuracy
def accuracy_metric(gt: Any, pred: Any) -> float:
    return 1.0 if gt == pred else 0.0

# Calculate precision, recall, and F1 for lists (assuming items are unique)
def precision_recall_f1(gt: List, pred: List) -> Dict[str, float]:
    gt_set, pred_set = set(gt), set(pred)
    true_positives = len(gt_set & pred_set)
    precision = true_positives / len(pred_set) if pred_set else 0
    recall = true_positives / len(gt_set) if gt_set else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return {"precision": precision, "recall": recall, "f1": f1_score}

# Calculate semantic distance for string answers using sequence matching
def semantic_similarity(gt: str, pred: str) -> float:
    return SequenceMatcher(None, gt, pred).ratio()  # Returns a ratio between 0 and 1

# Evaluate an answer based on the ground truth
def calculate_answer_metrics(gt: Any, pred: Any) -> Dict[str, float]:
    # Parse inputs
    gt, pred = parse_input(gt), parse_input(pred)
    
    # Initialize results
    results = {"accuracy": 0, "semantic_similarity": 0, "precision": 0, "recall":0, "f1":0}
    
    # Check type and apply appropriate metrics
    if isinstance(gt, float) and isinstance(pred, float):
        results["accuracy"] = 1.0 if isclose(gt, pred, rel_tol=1e-2) else 0.0  # Accuracy for floats with tolerance
        precision, recall = results["accuracy"], results["accuracy"]
        results.update({"precision": precision, "recall": recall, "f1": 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0})
        # results['bertscore'] = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
        results["semantic_similarity"] = semantic_similarity(str(gt), str(pred))
    elif isinstance(gt, int) and isinstance(pred, int):
        results["accuracy"] = 1.0 if isclose(gt, pred, rel_tol=1e-2) else 0.0  # Accuracy for floats with tolerance
        precision, recall = results["accuracy"], results["accuracy"]
        results.update({"precision": precision, "recall": recall, "f1": 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0})
        # results['bertscore'] = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
        results["semantic_similarity"] = semantic_similarity(str(gt), str(pred))


    elif isinstance(gt, str) and isinstance(pred, str):
        results["accuracy"] = accuracy_metric(gt, pred)
        precision, recall = results["accuracy"], results["accuracy"]
        results.update({"precision": precision, "recall": recall, "f1": 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0})
        # results['bertscore'] = score([pred], [gt], lang='en', verbose=True)
        results["semantic_similarity"] = semantic_similarity(gt, pred)

    
    elif isinstance(gt, list) and isinstance(pred, list):
        metrics = precision_recall_f1(gt, pred)
        results.update(metrics)
        results["accuracy"] = accuracy_metric(gt, pred)
        # results['bertscore'] = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
        similarity = semantic_similarity(f"{gt}", f"{pred}")
        results["semantic_similarity"] = similarity

    
    elif isinstance(gt, dict) and isinstance(pred, dict):
        gt_keys, pred_keys = list(gt.keys()), list(pred.keys())
        if type(gt[gt_keys[0]]) == dict:
            precision_recall_f1_results = [precision_recall_f1(gt[k].values(), pred[k].values()) for k in gt_keys]
            precision = sum([x['precision'] for x in precision_recall_f1_results])/len([x['precision'] for x in precision_recall_f1_results])
            recall = sum([x['recall'] for x in precision_recall_f1_results])/len([x['recall'] for x in precision_recall_f1_results])
            f1 =sum([x['f1'] for x in precision_recall_f1_results])/len([x['f1'] for x in precision_recall_f1_results])
            results.update({'precision': precision, 'recall':recall, 'f1': f1})  # Precision/Recall on keys

        elif type(gt[gt_keys[0]]) == list:
            precision_recall_f1_results = [precision_recall_f1(gt[k], pred[k]) for k in gt_keys]
            precision = sum([x['precision'] for x in precision_recall_f1_results])/len([x['precision'] for x in precision_recall_f1_results])
            recall = sum([x['recall'] for x in precision_recall_f1_results])/len([x['recall'] for x in precision_recall_f1_results])
            f1 =sum([x['f1'] for x in precision_recall_f1_results])/len([x['f1'] for x in precision_recall_f1_results])
            results.update({'precision': precision, 'recall':recall, 'f1': f1})  # Precision/Recall on keys
        

        results["accuracy"] = accuracy_metric(gt, pred)
        # Check semantic similarity for each key-value pair
        similarity = [semantic_similarity(str(gt[k]), str(pred.get(k, ""))) for k in gt_keys]
        results["semantic_similarity"] = sum(similarity) / len(similarity) if similarity else 0
        
        # results['bertscore'] = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
    p, r, f1 = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
    results.update({'bertscore_p': p.detach().cpu().tolist(),
    'bertscore_r': r.detach().cpu().tolist(),
    'bertscore_f1': f1.detach().cpu().tolist()})
    # print(results)


    return results

In [40]:
test_json = "{\"Zip\":{\"0\":96701,\"1\":96704,\"2\":96707,\"3\":96708,\"4\":96749,\"5\":96750,\"6\":96754,\"7\":96791,\"8\":96813,\"9\":96814,\"10\":96815,\"11\":96816,\"12\":96817,\"13\":96821,\"14\":96825,\"15\":96826},\"LoanCount\":{\"0\":1,\"1\":1,\"2\":4,\"3\":1,\"4\":1,\"5\":1,\"6\":1,\"7\":1,\"8\":1,\"9\":1,\"10\":1,\"11\":2,\"12\":1,\"13\":1,\"14\":1,\"15\":1}}"

In [16]:
test_json = parse_input(test_json)

In [29]:
test_json.keys()
test_json.get('Zip').values()

dict_values([96701, 96704, 96707, 96708, 96749, 96750, 96754, 96791, 96813, 96814, 96815, 96816, 96817, 96821, 96825, 96826])

In [45]:
# LLM-based history update solution
import importlib.util
import inspect
from typing import List
import requests
import json
import re
import difflib
from collections import Counter
# from spellchecker import SpellChecker
from datetime import datetime
import pandas as pd
import ast
import random
import logging 
# from history_update_problem.call_or import export_rows
from call_or import *

# from evaluation.data_compare import calculate_answer_metrics


def load_answer_dataset(datafile_path):
    """
    load json file, each line is a json dictionary

    datafile_path: str
    return: data:  list_of_dictionary
    """
    data = []
    with open(datafile_path, 'r') as f:
        for l in f:
            data.append(json.loads(l))
    return data

def eval_answers(answer_gt_path, answer_preds_llama):
    answer_gt = load_answer_dataset(answer_gt_path)
    answer_gt = pd.DataFrame(answer_gt)
    answer_preds_llama = load_answer_dataset(answer_preds_llama)
    answer_preds_llama = pd.DataFrame(answer_preds_llama)
    answer_compare = answer_gt.merge(answer_preds_llama[['pp_id', 'answer']], on='pp_id', how='left', suffixes=('_gt', '_preds'))

    results = []
    for i, row in answer_compare.iterrows():
        gt = row['answer_gt']
        preds = row['answer_preds']
        single_result = calculate_answer_metrics(gt, preds)
        single_result['pp_id'] = row['pp_id']
        results.append(single_result)
        # break
    return pd.DataFrame(results)

 
    

In [42]:
# @title single eexample
answer_gt_path = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_gt.json'
answer_gt = load_answer_dataset(answer_gt_path)
answer_gt = pd.DataFrame(answer_gt)
answer_preds_llama = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_llama3.1.json'
answer_preds_llama = load_answer_dataset(answer_preds_llama)
answer_preds_llama = pd.DataFrame(answer_preds_llama)
answer_compare = answer_gt.merge(answer_preds_llama[['pp_id', 'answer']], on='pp_id', how='left', suffixes=('_gt', '_preds'))

results = []
for i, row in answer_compare.iterrows():
    gt = row['answer_gt']
    preds = row['answer_preds']
    single_result = calculate_answer_metrics(gt, preds)
    single_result['pp_id'] = row['pp_id']
    print(gt, type(gt))
    print(preds, type(preds))
    results.append(single_result)
    break

pd.DataFrame(results)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 92.38it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 891.08it/s]


done in 0.02 seconds, 44.96 sentences/sec
22 <class 'int'>
22 <class 'int'>


,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
0,1.0,1.0,1.0,1.0,1.0,[1.0000004768371582],[1.0000004768371582],[1.0000004768371582],1


In [61]:
isclose(22,22, rel_tol=1e-2)

True

In [90]:
answer_gt_path = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_gt.json'
answer_preds_llama = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_llama3.1.json'
# model = 'gemma2'
# model = 'dirty'
# model = 'llama3.1'
model = 'mistral'
answer_preds = f'/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_{model}.json'
# answer_preds = f'/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_dirty.json'
eval_answer_results = eval_answers(answer_gt_path, answer_preds)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 99.18it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 712.11it/s]


...

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 35.05it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 763.16it/s]

done in 0.04 seconds, 25.14 sentences/sec


In [91]:
eval_answer_results.tail()

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
62,0.0,0.808511,0.0000,0.0000,0.0000,[0.9511916637420654],[0.8949240446090698],[0.9222003221511841],106
63,1.0,1.000000,1.0000,1.0000,1.0000,[0.9999998211860657],[0.9999998211860657],[0.9999998211860657],107
64,0.0,0.000000,0.0000,0.0000,0.0000,[0.874943733215332],[0.7801971435546875],[0.824858546257019],108
65,0.0,0.356275,0.0000,0.0000,0.0000,[0.9047510623931885],[0.8725006580352783],[0.8883332014083862],109
66,0.0,0.990506,0.8375,0.8375,0.8375,[0.9872503876686096],[0.988763689994812],[0.9880064129829407],110


In [92]:
model

'mistral'

In [93]:
eval_answer_results['bertscore_p'] = eval_answer_results['bertscore_p'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else None)
eval_answer_results['bertscore_r'] = eval_answer_results['bertscore_r'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else None)
eval_answer_results['bertscore_f1'] = eval_answer_results['bertscore_f1'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else None)


In [78]:
model

'llama3.1'

In [94]:
eval_answer_results.to_csv(f'/projects/bces/lanl2/LLM4DC/evaluation/{model}_answer_result.csv')
# eval_answer_results.to_csv(f'/projects/bces/lanl2/LLM4DC/evaluation/dirty_answer_result.csv')

In [168]:
model = 'llama3.1'

In [169]:
eval_answer_results = pd.read_csv(f'/projects/bces/lanl2/LLM4DC/evaluation/{model}_answer_result.csv')

In [170]:
eval_answer_results.describe()

,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000
mean,33.000000,0.447761,0.742243,0.615287,0.622741,0.610498,0.950686,0.947142,0.948540,58.074627
std,19.485037,0.501017,0.354760,0.459467,0.454278,0.451074,0.131489,0.135712,0.132678,35.047290
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,16.500000,0.000000,0.500585,0.000000,0.000000,0.000000,0.944181,0.942289,0.939296,31.500000
50%,33.000000,0.000000,0.995349,0.939394,0.950000,0.900000,0.999803,0.999803,0.999803,66.000000
75%,49.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,88.000000
max,66.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,110.000000


In [171]:
ppp_results = eval_answer_results[eval_answer_results['pp_id'] >= 62][eval_answer_results['pp_id'] <=91]

/tmp/ipykernel_1577642/2424205661.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_results = eval_answer_results[eval_answer_results['pp_id'] >= 62][eval_answer_results['pp_id'] <=91]


In [172]:
dish_results = eval_answer_results[eval_answer_results['pp_id'] >= 92]

In [173]:
chi_results = eval_answer_results[eval_answer_results['pp_id'] >= 31][eval_answer_results['pp_id'] <=61]

/tmp/ipykernel_1577642/3367146683.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_results = eval_answer_results[eval_answer_results['pp_id'] >= 31][eval_answer_results['pp_id'] <=61]


In [174]:
menu_results = eval_answer_results[eval_answer_results['pp_id'] < 31]

In [175]:
ppp_results.describe()

,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,39.500000,0.636364,0.876176,0.713120,0.721861,0.716450,0.975988,0.976735,0.976117,73.000000
std,6.493587,0.492366,0.219952,0.428810,0.422058,0.425785,0.047341,0.038679,0.041219,7.438638
min,29.000000,0.000000,0.266667,0.000000,0.000000,0.000000,0.800825,0.887636,0.857735,62.000000
25%,34.250000,0.000000,0.821821,0.397436,0.541667,0.452381,0.972412,0.972412,0.972412,67.250000
50%,39.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,72.500000
75%,44.750000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,77.750000
max,50.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,89.000000


In [176]:
dish_results.describe()

,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000
mean,58.500000,0.312500,0.803480,0.641427,0.627812,0.633603,0.903831,0.902170,0.902739,101.93750
std,4.760952,0.478714,0.348544,0.450720,0.447672,0.448163,0.246193,0.251014,0.248221,5.65059
min,51.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,92.00000
25%,54.750000,0.000000,0.815027,0.000000,0.000000,0.000000,0.929033,0.951412,0.940082,98.75000
50%,58.500000,0.000000,0.995650,0.887500,0.887500,0.887500,0.987596,0.990719,0.989155,102.50000
75%,62.250000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,106.25000
max,66.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,110.00000


In [177]:
menu_results.describe()

,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,16.000000,16.000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,0.375,0.561882,0.532670,0.596731,0.536699,0.973148,0.981845,0.977065,9.375000
std,4.760952,0.500,0.442968,0.484167,0.488279,0.465394,0.061415,0.038492,0.048293,5.667157
min,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.815612,0.885391,0.853846,1.000000
25%,3.750000,0.000,0.000000,0.000000,0.000000,0.000000,0.995815,0.996442,0.996128,4.750000
50%,7.500000,0.000,0.565951,0.636364,0.984375,0.616667,0.999808,0.999808,0.999808,8.500000
75%,11.250000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14.250000
max,15.000000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18.000000


In [103]:
chi_results.describe()

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,0.230769,0.616404,0.403846,0.357692,0.370879,0.945635,0.922062,0.933221,38.769231
std,0.438529,0.344286,0.473665,0.438675,0.442612,0.065947,0.095499,0.080116,6.260376
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.816275,0.734762,0.773377,31.000000
25%,0.000000,0.363636,0.000000,0.000000,0.000000,0.905686,0.861316,0.904977,34.000000
50%,0.000000,0.666667,0.000000,0.000000,0.000000,0.967485,0.963320,0.965398,38.000000
75%,0.000000,0.918919,1.000000,0.750000,0.750000,0.999451,0.999451,0.999451,41.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,52.000000


In [43]:
eval_answer_results = pd.DataFrame(eval_answer_results)

In [40]:
eval_answer_results['accuracy'].sum()

np.float64(21.0)

In [100]:
eval_answer_results.describe()

,accuracy,semantic_similarity,precision,recall,f1,pp_id
count,67.000000,67.0,67.000000,67.000000,67.000000,67.000000
mean,0.298507,0.0,0.444066,0.442419,0.442747,58.074627
std,0.461057,0.0,0.485182,0.484086,0.483975,35.047290
min,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.0,0.000000,0.000000,0.000000,31.500000
50%,0.000000,0.0,0.000000,0.000000,0.000000,66.000000
75%,1.000000,0.0,1.000000,1.000000,1.000000,88.000000
max,1.000000,0.0,1.000000,1.000000,1.000000,110.000000


## answer eval ttest


In [105]:
from scipy import stats

In [104]:
dirty_answer_result = pd.read_csv('/projects/bces/lanl2/LLM4DC/evaluation/dirty_answer_result.csv')
gemma2_answer_result = pd.read_csv('/projects/bces/lanl2/LLM4DC/evaluation/gemma2_answer_result.csv')
llama_answer_result = pd.read_csv('/projects/bces/lanl2/LLM4DC/evaluation/llama3.1_answer_result.csv')
mistral_answer_result = pd.read_csv('/projects/bces/lanl2/LLM4DC/evaluation/mistral_answer_result.csv')

In [106]:
answer_result = llama_answer_result.merge(gemma2_answer_result, on='pp_id', how='left', suffixes=('_llama', '_gemma2')).merge(mistral_answer_result, on='pp_id', how='left', suffixes=('', '_mistral'))

In [107]:
answer_result = answer_result.merge(dirty_answer_result, on='pp_id', how='left', suffixes=('', '_dirty'))

In [108]:
answer_result.head()

,Unnamed: 0_llama,accuracy_llama,semantic_similarity_llama,precision_llama,recall_llama,f1_llama,bertscore_p_llama,bertscore_r_llama,bertscore_f1_llama,pp_id,...,bertscore_f1,Unnamed: 0_dirty,accuracy_dirty,semantic_similarity_dirty,precision_dirty,recall_dirty,f1_dirty,bertscore_p_dirty,bertscore_r_dirty,bertscore_f1_dirty
0,0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1,...,1.000000,0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
1,1,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,2,...,1.000000,1,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000
2,2,0.0,0.0,0.0,0.0,0.0,0.999803,0.999803,0.999803,3,...,0.999477,2,0.0,0.0,0.0,0.0,0.0,0.998309,0.998309,0.998309
3,3,0.0,0.0,0.0,0.0,0.0,0.999268,0.999268,0.999268,4,...,0.999349,3,0.0,0.5,0.0,0.0,0.0,0.999349,0.999349,0.999349
4,4,0.0,0.0,0.0,0.0,0.0,0.999813,0.999813,0.999813,5,...,0.999813,4,0.0,0.0,0.0,0.0,0.0,0.999813,0.999813,0.999813


In [110]:
metric_names = ['accuracy','semantic_similarity', 'precision', 'recall', 'f1', 'bertscore_p', 'bertscore_r','bertscore_f1']


In [111]:
ttest_result = []
for m in metric_names:
    model_list = ['llama', 'gemma2', 'mistral', 'dirty']
    llama_result = stats.ttest_rel(answer_result[f'{m}_dirty'].values, answer_result[f'{m}_llama'].values)
    gemma_result = stats.ttest_rel(answer_result[f'{m}_dirty'].values, answer_result[f'{m}_gemma2'].values)
    # print(answer_result[f'{m}_llama'].values)
    mistral_result = stats.ttest_ind(answer_result[f'{m}_dirty'].values, answer_result[f'{m}'].values)
    ttest_result.append({f'{m}':[llama_result.pvalue.item(), gemma_result.pvalue.item(), mistral_result.pvalue.item()]})
    # print(gemma_result)
    # print(mistral_result)
    # break

In [115]:
def ttest_answer(answer_result):
    ttest_result = []
    for m in metric_names:
        model_list = ['llama', 'gemma2', 'mistral', 'dirty']
        llama_result = stats.ttest_rel(answer_result[f'{m}_dirty'].values, answer_result[f'{m}_llama'].values)
        gemma_result = stats.ttest_rel(answer_result[f'{m}_dirty'].values, answer_result[f'{m}_gemma2'].values)
        # print(answer_result[f'{m}_llama'].values)
        mistral_result = stats.ttest_ind(answer_result[f'{m}_dirty'].values, answer_result[f'{m}'].values)
        ttest_result.append({f'{m}':[llama_result.pvalue.item(), gemma_result.pvalue.item(), mistral_result.pvalue.item()]})
    combined_data = {k: v for d in ttest_result for k, v in d.items()}

    ttest_result = pd.DataFrame(combined_data)
    return ttest_result

In [113]:
combined_data = {k: v for d in ttest_result for k, v in d.items()}

ttest_result = pd.DataFrame(combined_data)

In [114]:
ttest_result

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,0.409465,0.016196,0.010599,0.004224,0.009252,0.971096,0.945330,0.993090
1,0.798445,0.502049,0.553362,0.589685,0.546391,0.383719,0.826350,0.769054
2,0.373586,0.966990,0.920003,0.790593,0.844258,0.873258,0.592786,0.701420


In [116]:
ppp_results = answer_result[answer_result['pp_id'] >= 62][answer_result['pp_id'] <=91]
ppp_ttest = ttest_answer(ppp_results)

/tmp/ipykernel_1577642/3278763139.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_results = answer_result[answer_result['pp_id'] >= 62][answer_result['pp_id'] <=91]


In [118]:
ppp_ttest

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,0.665194,0.071289,0.432985,0.521864,0.468124,0.332356,0.588604,0.430670
1,0.328695,0.188103,0.291698,0.204980,0.253631,0.164633,0.602721,0.312134
2,0.555640,0.991541,0.584520,0.548433,0.568577,0.751530,0.622030,0.671129


In [119]:
dish_result= answer_result[answer_result['pp_id'] >= 92]
dish_ttest = ttest_answer(dish_result)
dish_ttest

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,0.580471,0.066705,0.040415,0.044881,0.042673,0.774325,0.862575,0.817219
1,0.718301,0.462110,0.523248,0.523248,0.523248,0.672301,0.897460,0.910012
2,0.680930,0.445805,0.787698,0.787698,0.787698,0.714108,0.377144,0.505227


In [121]:
menu_results = answer_result[answer_result['pp_id'] < 31]
menu_ttest = ttest_answer(menu_results)
menu_ttest

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,0.580471,0.599889,0.890791,0.141362,0.825616,0.449021,0.794551,0.713241
1,0.580471,0.288723,0.839887,0.988945,0.949398,0.283264,0.347259,0.314584
2,0.481310,0.919102,0.924996,0.944534,0.935819,0.629457,0.729884,0.688068


In [122]:
chi_results = answer_result[answer_result['pp_id'] >= 31][answer_result['pp_id'] <=61]
chi_ttest = ttest_answer(chi_results)
chi_ttest

/tmp/ipykernel_1577642/3533590064.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_results = answer_result[answer_result['pp_id'] >= 31][answer_result['pp_id'] <=61]


,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,0.165407,0.196659,0.052011,0.087528,0.061788,0.212637,0.808143,0.504440
1,0.337049,0.875186,0.375308,0.433098,0.391871,0.642579,0.931535,0.876010
2,1.000000,0.525309,0.372043,0.559367,0.474808,0.523184,0.682367,0.608927


In [123]:
ppp_results = answer_result[answer_result['pp_id'] >= 62][answer_result['pp_id'] <=91]
ppp_ttest = ttest_answer(ppp_results)
ppp_ttest

/tmp/ipykernel_1577642/2612646877.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_results = answer_result[answer_result['pp_id'] >= 62][answer_result['pp_id'] <=91]


,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,0.665194,0.071289,0.432985,0.521864,0.468124,0.332356,0.588604,0.430670
1,0.328695,0.188103,0.291698,0.204980,0.253631,0.164633,0.602721,0.312134
2,0.555640,0.991541,0.584520,0.548433,0.568577,0.751530,0.622030,0.671129


In [4]:
def json_to_custom_string(data):
    # Serialize JSON with indentation for readability within each nested structure
    json_str = json.dumps(data, indent=None, separators=(r'')
    
    # Insert newline only after each top-level key-value pair
    # This keeps inner lists and dicts intact without adding extra line breaks
    formatted_str = json_str[1:-1].replace('", "', '",\n"')  # Exclude outer braces for custom formatting
    return f"{{\n{formatted_str}\n}}"


In [ ]:
json.dumps(data, indent=1, sepera)

In [11]:
data = {'table_caption': 'A mix of simple bibliographic description of the menus', 'columns': ['id', 'name', 'sponsor', 'event', 'venue', 'place', 'physical_description', 'occasion', 'notes', 'call_number', 'keywords', 'language', 'date', 'location', 'location_type', 'currency', 'currency_symbol', 'status', 'page_count', 'dish_count'], 'table_column_priority': [['id', '12579', '25121', '21960'], ['name', '', '', ''], ['sponsor', 'TRUSTEES OF THE MISSOURI BOTANICAL GARDEN', 'HOLLAND HOUSE', 'BATTERY PARK HOTEL'], ['event', '11TH ANNUAL BANQUET', 'LUNCHEON', 'CHRISTMAS DINNER'], ['venue', 'PROF;', 'COMMERCIAL', 'COMMERCIAL'], ['place', 'SOUTHERN HOTEL,ST. LOUIS,MO.', '', 'ASHVILLE, NC'], ['physical_description', 'BROADSIDE; ILLUS; 5.5 X 8.75;', 'CARD;6X8.75;', 'BOOKLET; ILLUS; COL; 6 X 8;'], ['occasion', 'ANNUAL', 'DAILY;', 'RELIGIOUS HOLIDAY'], ['notes', 'WINES LISTED FOR EACH COURSE;', 'ENGLISH ON ONE SIDE,FRENCH ON ONE SIDE;', 'PRINTED ON PARCHMENT-LIKE PAPER; RED LION; DRAWING OF HOTEL; TIED WITH TWINE AND SEALING WAX; MOST OF BOOKLET CONSISTS OF INFORMATION AND PRAISE ABOUT THE HOTEL;'], ['call_number', '1900-2627', '1900-517', '1898-432'], ['keywords', '', '', ''], ['language', '', '', ''], ['date', '1900-03-31', '1900-01-25', '1898-12-25'], ['location', 'Trustees Of The Missouri Botanical Garden', 'Holland House', 'Battery Park Hotel'], ['location_type', '', '', ''], ['currency', '', 'Dollars', ''], ['currency_symbol', '', '$', ''], ['status', 'complete', 'complete', 'complete'], ['page_count', '2', '2', '16'], ['dish_count', '22.0', '546.0', '28.0']]}


In [27]:
data_str = '\{\n'
for key, value in data.items():
    if key in ['table_caption', 'columns']:
        data_str += f'{key}: {value},\n'
    else:
        data_str += key + ': [\n'
        for l in value:
            data_str += f'  {l},\n'
data_str += '  ]\n}'
    


In [31]:
type(data_str)

str

In [29]:
pprint.pprint(data_str)

('{\n'
 'table_caption: A mix of simple bibliographic description of the menus,\n'
 "columns: ['id', 'name', 'sponsor', 'event', 'venue', 'place', "
 "'physical_description', 'occasion', 'notes', 'call_number', 'keywords', "
 "'language', 'date', 'location', 'location_type', 'currency', "
 "'currency_symbol', 'status', 'page_count', 'dish_count'],\n"
 'table_column_priority: [\n'
 "  ['id', '12579', '25121', '21960'],\n"
 "  ['name', '', '', ''],\n"
 "  ['sponsor', 'TRUSTEES OF THE MISSOURI BOTANICAL GARDEN', 'HOLLAND HOUSE', "
 "'BATTERY PARK HOTEL'],\n"
 "  ['event', '11TH ANNUAL BANQUET', 'LUNCHEON', 'CHRISTMAS DINNER'],\n"
 "  ['venue', 'PROF;', 'COMMERCIAL', 'COMMERCIAL'],\n"
 "  ['place', 'SOUTHERN HOTEL,ST. LOUIS,MO.', '', 'ASHVILLE, NC'],\n"
 "  ['physical_description', 'BROADSIDE; ILLUS; 5.5 X 8.75;', 'CARD;6X8.75;', "
 "'BOOKLET; ILLUS; COL; 6 X 8;'],\n"
 "  ['occasion', 'ANNUAL', 'DAILY;', 'RELIGIOUS HOLIDAY'],\n"
 "  ['notes', 'WINES LISTED FOR EACH COURSE;', 'ENGLISH ON ON

In [10]:
import pandas as pd

In [7]:
import pprint

In [8]:
pprint.pprint(json_to_custom_string(data))

('{\n'
 '"table_caption": "A mix of simple bibliographic description of the '
 'menus","columns": '
 '["id","name","sponsor","event","venue","place","physical_description","occasion","notes","call_number","keywords","language","date","location","location_type","currency","currency_symbol","status","page_count","dish_count"],"table_column_priority": '
 '[["id","12579","25121","21960"],["name","","",""],["sponsor","TRUSTEES OF '
 'THE MISSOURI BOTANICAL GARDEN","HOLLAND HOUSE","BATTERY PARK '
 'HOTEL"],["event","11TH ANNUAL BANQUET","LUNCHEON","CHRISTMAS '
 'DINNER"],["venue","PROF;","COMMERCIAL","COMMERCIAL"],["place","SOUTHERN '
 'HOTEL,ST. LOUIS,MO.","","ASHVILLE, NC"],["physical_description","BROADSIDE; '
 'ILLUS; 5.5 X 8.75;","CARD;6X8.75;","BOOKLET; ILLUS; COL; 6 X '
 '8;"],["occasion","ANNUAL","DAILY;","RELIGIOUS HOLIDAY"],["notes","WINES '
 'LISTED FOR EACH COURSE;","ENGLISH ON ONE SIDE,FRENCH ON ONE SIDE;","PRINTED '
 'ON PARCHMENT-LIKE PAPER; RED LION; DRAWING OF HOTEL; TIED WI

In [28]:
paragraph = "[    {'from': ['Luncheon', 'lunch'], 'to': 'Luncheon'},    {'from': ['Dinner'], 'to': 'Dinner'},  # No change needed for Dinner    {'from': ['Breakfast'], 'to': 'Breakfast'},  # No change needed for Breakfast    {'from': ['', 'Unknown', None], 'to': 'Unknown'},  # Replace missing values with Unknown    {'from': ['Banquet', 'Annual Banquet'], 'to': 'Annual Banquet'},    {'from': ['Tiffin'], 'to': 'Tiffin'}  # No change needed for Tiffin]"

In [3]:
raw_string = """```python
[
    {'from': ['HONOLULU', 'Honolulu'], 'to': 'Honolulu'},
    {'from': ['KANEOHE'], 'to': 'Kaneohe'},  # corrected spelling
    {'from': ['KIHEI'], 'to': 'Kihei'},  # corrected spelling
    {'from': ['Kailua Kona', 'KAUNAKAKAI', 'WAIMANALO', 'Waimea', 'kalaheo', 'Kahului'], 'to': 'Unknown'},  # unknown cities
    {'from': ['HONOLULUulu'], 'to': 'Honolulu'}  # corrected spelling
]
```"""

In [4]:
result = re.findall(r'(\[(:?\n?.*\n?)*\])', raw_string, re.DOTALL)

In [5]:
if result:
        for r in result:
            raw_string = r[0]

In [6]:
raw_string

"[\n    {'from': ['HONOLULU', 'Honolulu'], 'to': 'Honolulu'},\n    {'from': ['KANEOHE'], 'to': 'Kaneohe'},  # corrected spelling\n    {'from': ['KIHEI'], 'to': 'Kihei'},  # corrected spelling\n    {'from': ['Kailua Kona', 'KAUNAKAKAI', 'WAIMANALO', 'Waimea', 'kalaheo', 'Kahului'], 'to': 'Unknown'},  # unknown cities\n    {'from': ['HONOLULUulu'], 'to': 'Honolulu'}  # corrected spelling\n]"

In [7]:
eval(raw_string)

[{'from': ['HONOLULU', 'Honolulu'], 'to': 'Honolulu'},
 {'from': ['KANEOHE'], 'to': 'Kaneohe'},
 {'from': ['KIHEI'], 'to': 'Kihei'},
 {'from': ['Kailua Kona',
   'KAUNAKAKAI',
   'WAIMANALO',
   'Waimea',
   'kalaheo',
   'Kahului'],
  'to': 'Unknown'},
 {'from': ['HONOLULUulu'], 'to': 'Honolulu'}]

In [26]:
paragraph = re.sub(r'#.*\{$', '{', paragraph, flags=re.MULTILINE)


In [29]:
result = re.findall(r'(\[(:?\n?(^#.*\{|]}).*\n?)*\])', paragraph, re.DOTALL)


In [30]:
line = "{'city':['hhhh'] #????ddfgtecg"

[]

In [5]:
input = '''
[{'foo':'bar'} # 000asd
]'''
eval(input)


[{'foo': 'bar'}]

In [27]:
paragraph

"[    {'from': ['Luncheon', 'lunch'], 'to': 'Luncheon'},    {'from': ['Dinner'], 'to': 'Dinner'},  # No change needed for Dinner    {'from': ['Breakfast'], 'to': 'Breakfast'},  # No change needed for Breakfast    {'from': ['', 'Unknown', None], 'to': 'Unknown'},  # Replace missing values with Unknown    {'from': ['Banquet', 'Annual Banquet'], 'to': 'Annual Banquet'},    {'from': ['Tiffin'], 'to': 'Tiffin'}  # No change needed for Tiffin]"

In [1]:
import pandas as pd
import json

def df_to_table_format(df, table_caption):
    # Extract column headers and index to form the first row in the table
    columns = ["competition"] + df.columns.tolist()
    
    # Prepare rows by combining index and corresponding row values in the DataFrame
    table_column_priority = [columns]  # Start with header row
    for idx, row in df.iterrows():
        table_column_priority.append([idx] + row.tolist())
    
    # Create the dictionary in the required format
    table_data = {
        "table_caption": table_caption,
        "columns": columns,
        "table_column_priority": table_column_priority
    }
    
    # Convert to JSON string with indentation for readability
    return json.dumps(table_data, indent=2)

# Example DataFrame
data = {
    "total matches": ["55", "2", "5"],
    "cardiff win": ["19", "0", "2"],
    "draw": ["16", "27", "0"],
    "swansea win": ["20", "2", "3"]
}
index = ["league", "fa cup", "league cup"]

df = pd.DataFrame(data, index=index)

# Convert DataFrame to the specified format and print
table_format_json = df_to_table_format(df, "south wales derby")
print(table_format_json)


{
  "table_caption": "south wales derby",
  "columns": [
    "competition",
    "total matches",
    "cardiff win",
    "draw",
    "swansea win"
  ],
  "table_column_priority": [
    [
      "competition",
      "total matches",
      "cardiff win",
      "draw",
      "swansea win"
    ],
    [
      "league",
      "55",
      "19",
      "16",
      "20"
    ],
    [
      "fa cup",
      "2",
      "0",
      "27",
      "2"
    ],
    [
      "league cup",
      "5",
      "2",
      "0",
      "3"
    ]
  ]
}


In [3]:
import pandas as pd
import json

def df_to_table_format(df, table_caption):
    # Extract column headers and index to form the first row in the table
    columns = ["competition"] + df.columns.tolist()
    
    # Prepare rows by combining index and corresponding row values in the DataFrame
    table_column_priority = [columns]  # Start with header row
    for idx, row in df.iterrows():
        table_column_priority.append([idx] + row.tolist())
    
    # Create the dictionary in the required format
    table_data = {
        "table_caption": table_caption,
        "columns": columns,
        "table_column_priority": table_column_priority
    }
    
    # Format 'columns' as a compact list and use json.dumps for the rest
    columns_str = f'"columns": {json.dumps(columns)}'
    table_column_priority_str = f'"table_column_priority": {json.dumps(table_column_priority)}'
    caption_str = f'"table_caption": "{table_caption}"'
    
    # Concatenate each component into a final JSON format
    final_output = f'{{\n  {caption_str},\n  {columns_str},\n  {table_column_priority_str}\n}}'
    
    return final_output

# Example DataFrame
data = {
    "total matches": ["55", "2", "5"],
    "cardiff win": ["19", "0", "2"],
    "draw": ["16", "27", "0"],
    "swansea win": ["20", "2", "3"]
}
index = ["league", "fa cup", "league cup"]

df = pd.DataFrame(data, index=index)

# Convert DataFrame to the specified format and print
table_format_json = df_to_table_format(df, "south wales derby")
print(table_format_json)


{
  "table_caption": "south wales derby",
  "columns": ["competition", "total matches", "cardiff win", "draw", "swansea win"],
  "table_column_priority": [["competition", "total matches", "cardiff win", "draw", "swansea win"], ["league", "55", "19", "16", "20"], ["fa cup", "2", "0", "27", "2"], ["league cup", "5", "2", "0", "3"]]
}


In [4]:
import pandas as pd
import json

def df_to_table_format(df, table_caption):
    # Prepare rows for table_column_priority
    table_column_priority = []
    
    # The first list is the competition names (the DataFrame index)
    table_column_priority.append(["competition"] + df.index.tolist())
    
    # Append each DataFrame column values to table_column_priority
    for col in df.columns:
        table_column_priority.append([col] + df[col].tolist())
    
    # Create the final dictionary
    table_data = {
        "table_caption": table_caption,
        "columns": ["competition"] + df.columns.tolist(),
        "table_column_priority": table_column_priority
    }
    
    return table_data

# Example DataFrame
data = {
    "total matches": ["55", "2", "5"],
    "cardiff win": ["19", "0", "2"],
    "draw": ["16", "27", "0"],
    "swansea win": ["20", "2", "3"]
}
index = ["league", "fa cup", "league cup"]

df = pd.DataFrame(data, index=index)

# Convert DataFrame to the specified format
table_format = df_to_table_format(df, "south wales derby")

# Print the output in the desired format
print(json.dumps(table_format, indent=2))


{
  "table_caption": "south wales derby",
  "columns": [
    "competition",
    "total matches",
    "cardiff win",
    "draw",
    "swansea win"
  ],
  "table_column_priority": [
    [
      "competition",
      "league",
      "fa cup",
      "league cup"
    ],
    [
      "total matches",
      "55",
      "2",
      "5"
    ],
    [
      "cardiff win",
      "19",
      "0",
      "2"
    ],
    [
      "draw",
      "16",
      "27",
      "0"
    ],
    [
      "swansea win",
      "20",
      "2",
      "3"
    ]
  ]
}
